# Scrape Most Recent Practice Races
Jack Wilson
9/21/2025

# Import Modules

In [ ]:
import pandas as pd
import time, random

from datetime import datetime

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait

In [14]:
pd.set_option('display.max_columns', None)
pd.set_option("display.max_colwidth", None)

# Scrape Practice

In [ ]:
# Establish current year
year = datetime.now().year
#year = 2026

# Establish URL and round list
race_url = []
rounds = []
r = 1

# Establish web browser
browser = webdriver.Chrome()
browser.maximize_window()

while True:
    # Get URL
    url = 'https://www.formula1.com/en/results/' + str(year) + '/races'
    print(year)
    print(url)
    browser.get(url)
    time.sleep(random.uniform(1,2))
    
    # Try to get data for that that year, or go to the year prior and try again
    try:
        # Find the table
        table = browser.find_elements(By.TAG_NAME, 'table')
        for tr in table:
            # Find the table rows
            rows = tr.find_elements(By.TAG_NAME, 'tr')[1:]
            for row in rows:
                # Find the table data
                cells = row.find_elements(By.TAG_NAME, 'td')
                
                # Get the race url and round number
                link = cells[0].find_element(By.TAG_NAME, 'a')
                race_url.append(link.get_attribute('href'))
                rounds.append(r)
                r += 1
        
        # Break the loop if data is found
        break
    # If no data is found subtract 1 from the year and try again
    except:
        year -= 1
        # If the year is less than 1950, break the loop so it isn't infinite
        if year < 1949:
            break
        pass

# Extract the country from the last race URL and create practice session URLs
last_race = race_url[-1]
last_round = rounds[-1]

race_parts = last_race.split('/')
last_race_country = race_parts[-2]

# Set the practice number to start at 1
p = 1

while True:
    # Get the practice URL
    practice_url = last_race.replace('/race-result', '/practice/') + str(p)
    browser.get(practice_url)
    print(practice_url)
    time.sleep(random.uniform(1,2))
    
    try:
        # Find the table
        table = browser.find_elements(By.TAG_NAME, 'table')
        for tr in table:
            # Find the table rows
            rows = tr.find_elements(By.TAG_NAME, 'tr')[1:]
            for row in rows:
                # Find the table data
                cells = row.find_elements(By.TAG_NAME, 'td')
                
                # Get the race url and round number
                link = cells[0].find_element(By.TAG_NAME, 'a')
                race_url.append(link.get_attribute('href'))
        
        p += 1
        if p > 3:
            break
    except:
        p += 1
        if p > 3:
            break



browser.close()


2025
https://www.formula1.com/en/results/2025/races
https://www.formula1.com/en/results/2025/races/1269/azerbaijan/race-result
17


https://www.formula1.com/en/results/2025/races/1269/azerbaijan/practice/1
https://www.formula1.com/en/results/2025/races/1269/azerbaijan/practice/2
https://www.formula1.com/en/results/2025/races/1269/azerbaijan/practice/3
https://www.formula1.com/en/results/2025/races/1269/azerbaijan/practice/4


In [19]:
p1 = pd.DataFrame({
    'url': race_url,
    'round': rounds
})

In [20]:
p1

,url,round
0,https://www.formula1.com/en/results/2025/races/1254/australia/race-result,1
1,https://www.formula1.com/en/results/2025/races/1255/china/race-result,2
2,https://www.formula1.com/en/results/2025/races/1256/japan/race-result,3
3,https://www.formula1.com/en/results/2025/races/1257/bahrain/race-result,4
4,https://www.formula1.com/en/results/2025/races/1258/saudi-arabia/race-result,5
5,https://www.formula1.com/en/results/2025/races/1259/miami/race-result,6
6,https://www.formula1.com/en/results/2025/races/1260/emilia-romagna/race-result,7
7,https://www.formula1.com/en/results/2025/races/1261/monaco/race-result,8
8,https://www.formula1.com/en/results/2025/races/1262/spain/race-result,9
9,https://www.formula1.com/en/results/2025/races/1263/canada/race-result,10
